In [1]:

#path="./BasicsMusicalInstrumClassifi/audio/london_phill_dataset_multi/"

In [2]:
import librosa
import os,fnmatch
import numpy as np
import pandas as pd

import librosa.display
import IPython.display as ipd

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import glob
import os

# Define the path with the wildcard pattern
path = "BasicsMusicalInstrumClassifi/audio/london_phill_dataset_multi/**/*.mp3"

# Use glob to find all .mp3 files matching the pattern
files = glob.glob(path, recursive=True)

print("found %d audio files in %s" % (len(files), path))


In [5]:
# Get Labels
labels =[]
classes=['cello','flute','oboe','saxaphone','trumpet','viola']
color_dict={'cello':'blue', 'flute':'red', 'oboe':'green', 'saxaphone':'black', 'trumpet':'magenta', 'viola':'yellow'}
color_list=[]
for filename in files:
    for name in classes:
        if fnmatch.fnmatchcase(filename, '*'+name+'*'):
            labels.append(name)
            color_list.append(color_dict[name])
            break
    else:
        labels.append('other')

In [ ]:
# Encode Labels
labelencoder = LabelEncoder()
labelencoder.fit(labels)
print(len(labelencoder.classes_), "classes:", ", ".join(list(labelencoder.classes_)))
classes_num = labelencoder.transform(labels)

In [7]:
# Parameters
# Signal Processing Parameters
fs = 44100         # Sampling Frequency
n_fft = 2048       # length of the FFT window
hop_length = 512   # Number of samples between successive frames
n_mels = 128       # Number of Mel bands
n_mfcc = 13        # Number of MFCCs

# Machine Learning Parameters
testset_size = 0.25 #Percentage of data for Testing
n_neighbors=1       # Number of neighbors for kNN Classifier

In [8]:
# Define Function to Calculate MFCC, Delta_MFCC and Delta2_MFCC
def get_features(y, sr=fs):
    S = librosa.feature.melspectrogram(y=y, sr=fs, n_mels=n_mels)
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(S), n_mfcc=n_mfcc)
    feature_vector = np.mean(mfcc,1)
    #feature_vector = (feature_vector-np.mean(feature_vector))/np.std(feature_vector)
    return feature_vector

In [ ]:
feature_vectors = []
sound_paths = []
fs = 22050  # Example sampling rate

for i, f in enumerate(files):
    print("get %d of %d = %s" % (i + 1, len(files), f))
    try:
        y, sr = librosa.load(f, sr=fs)
        y /= y.max()  # Normalize
        if len(y) < 2:
            print("Error loading %s" % f)
            continue
        feat = get_features(y, sr)
        feature_vectors.append(feat)
        sound_paths.append(f)
    except Exception as e:
        print("Error loading %s. Error: %s" % (f, e))

print("Calculated %d feature vectors" % len(feature_vectors))

In [ ]:
print(len(feature_vectors))

In [ ]:

# Scale features using Standard Scaler
scaler = StandardScaler()
scaled_feature_vectors = scaler.fit_transform(np.array(feature_vectors))
print("Feature vectors shape:",scaled_feature_vectors.shape)

In [12]:
# Create Train and Test Set
splitter = StratifiedShuffleSplit(n_splits=1, test_size=testset_size, random_state=0)
splits = splitter.split(scaled_feature_vectors, classes_num)
for train_index, test_index in splits:
    train_set = scaled_feature_vectors[train_index]
    test_set = scaled_feature_vectors[test_index]
    train_classes = classes_num[train_index]
    test_classes = classes_num[test_index]

In [ ]:
# Check Set Shapes
print("train_set shape:",train_set.shape)
print("test_set shape:",test_set.shape)
print("train_classes shape:",train_classes.shape)
print("test_classes shape:",test_classes.shape)

In [14]:
# KNN Classifier
n_neighbors=1
model_knn = KNeighborsClassifier(n_neighbors=n_neighbors)

In [15]:
# kNN
model_knn.fit(train_set, train_classes);

In [16]:
# Predict using the Test Set
predicted_labels = model_knn.predict(test_set)

In [ ]:



print(labelencoder.inverse_transform(predicted_labels))

In [ ]:
#Evaluation
# Recall - the ability of the classifier to find all the positive samples
print("Recall: ", recall_score(test_classes, predicted_labels,average=None))

# Precision - The precision is intuitively the ability of the classifier not to 
#label as positive a sample that is negative
print("Precision: ", precision_score(test_classes, predicted_labels,average=None))

# F1-Score - The F1 score can be interpreted as a weighted average of the precision 
#and recall
print("F1-Score: ", f1_score(test_classes, predicted_labels, average=None))

# Accuracy - the number of correctly classified samples
print("Accuracy: %.2f  ," % accuracy_score(test_classes, predicted_labels,normalize=True), accuracy_score(test_classes, predicted_labels,normalize=False) )
print("Number of samples:",test_classes.shape[0])

# Precision - The precision is intuitively the ability of the classifier not to label as positive a sample that is negative
# Recall - the ability of the classifier to find all the positive samples
# F1-Score - The F1 score can be interpreted as a weighted average of the precision and recall

In [ ]:
print(classification_report(test_classes, predicted_labels))

In [20]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(test_classes, predicted_labels)
np.set_printoptions(precision=2)

In [ ]:
import itertools

# Define the plot_confusion_matrix function
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot non-normalized confusion matrix
plt.figure(figsize=(18,9))
plot_confusion_matrix(cnf_matrix, classes=labelencoder.classes_,
                      title='Confusion matrix, without normalization')

In [22]:
# Find wrong predicted samples indexes
#wrong_predictions = [i for i, (e1, e2) in enumerate(zip(test_classes, predicted_labels)) if e1 != e2]
wrong_predictions = [i for i, (e1, e2) in enumerate(zip(predicted_labels,test_classes)) if e1 != e2]

In [ ]:
# Find wrong predicted audio files
print(np.array(labels)[test_index[wrong_predictions]])
print(predicted_labels[wrong_predictions].T)
print(labelencoder.inverse_transform(predicted_labels[wrong_predictions]))
print(np.array(files)[test_index[wrong_predictions]])

In [24]:
#choosing a k value

error_rate = []

for i in range(1,20):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(train_set, train_classes)
    pred_i = knn.predict(test_set)
    error_rate.append(np.mean(pred_i != test_classes))
    # wherever the pred value is not matching the test class value it will find the mean of that which will give the error rate.

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,20),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

From the plot we see that the least stable error rate occurs around k =7 hence we will use k = 7 as the best model

In [26]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(train_set, train_classes)
# Predict using the Test Set
predicted_labels = knn.predict(test_set)

In [ ]:
# Accuracy - the number of correctly classified samples
print("Accuracy: %.2f  ," % accuracy_score(test_classes, predicted_labels,normalize=True), accuracy_score(test_classes, predicted_labels,normalize=False) )
print("Number of samples:",test_classes.shape[0])

In [ ]:
#Evaluation
# Recall - the ability of the classifier to find all the positive samples
print("Recall: ", recall_score(test_classes, predicted_labels,average=None))

# Precision - The precision is intuitively the ability of the classifier not to 
#label as positive a sample that is negative
print("Precision: ", precision_score(test_classes, predicted_labels,average=None))

# F1-Score - The F1 score can be interpreted as a weighted average of the precision 
#and recall
print("F1-Score: ", f1_score(test_classes, predicted_labels, average=None))

# Accuracy - the number of correctly classified samples
print("Accuracy: %.2f  ," % accuracy_score(test_classes, predicted_labels,normalize=True), accuracy_score(test_classes, predicted_labels,normalize=False) )
print("Number of samples:",test_classes.shape[0])

In [ ]:
print(classification_report(test_classes, predicted_labels))

In [30]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(test_classes, predicted_labels)
np.set_printoptions(precision=2)

In [ ]:
# Plot non-normalized confusion matrix
plt.figure(figsize=(18,9))
plot_confusion_matrix(cnf_matrix, classes=labelencoder.classes_,
                      title='Confusion matrix, without normalization')

In [32]:
# Find wrong predicted samples indexes
wrong_predictions = [i for i, (e1, e2) in enumerate(zip(predicted_labels,test_classes)) if e1 != e2]

In [ ]:
# Find wrong predicted audio files
print(np.array(labels)[test_index[wrong_predictions]])
print(predicted_labels[wrong_predictions].T)
print(labelencoder.inverse_transform(predicted_labels[wrong_predictions]))
print(np.array(files)[test_index[wrong_predictions]])

In [ ]:
#test_audio_path = '/home/dani/Desktop/proiectdl/BasicsMusicalInstrumClassifi/audio/london_phill_dataset_multi/sax/saxophone_A3_1_forte_normal.mp3'

test_audio_path = 'BasicsMusicalInstrumClassifi/audio/london_phill_dataset_multi/cello/cello_Gs4_025_pianissimo_arco-normal.mp3'
#test_audio_path = '/home/dani/Desktop/proiectdl/BasicsMusicalInstrumClassifi/audio/london_phill_dataset_multi/oboe/oboe_A5_1_forte_normal.mp3'
# Define function for extracting MFCC features
def extract_features(audio_path):
    y, sr = librosa.load(audio_path, sr=44100)
    # Normalize the audio
    y /= np.max(np.abs(y))
    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs, axis=1)
    return mfccs_mean

# Function to predict the class of a single audio file
def predict_audio_class(audio_path, model_knn, labelencoder, scaler):
    # Extract features from the audio file
    features = extract_features(audio_path)
    
    # Reshape features to fit the model input
    features_reshaped = features.reshape(1, -1)
    
    # Scale the features
    scaled_features = scaler.transform(features_reshaped)
    
    # Predict the label using the trained model
    predicted_label = model_knn.predict(scaled_features)
    
    # Convert label back to class name using label encoder
    predicted_class_name = labelencoder.inverse_transform(predicted_label)
    
    return predicted_class_name[0]

# Example usag
predicted_class = predict_audio_class(test_audio_path, model_knn, labelencoder, scaler)


import matplotlib.image as mpimg

# Path to the folder containing instrument images
image_folder = "instruments/"

# Function to display the instrument image
def display_instrument_image(predicted_class):
  
    # Map the instrument name to the corresponding image file
    image_path = os.path.join(image_folder, f"{predicted_class}.jpeg")  
    
    if os.path.exists(image_path):
        try:
            # Load and display the image
            img = mpimg.imread(image_path)
            plt.figure(figsize=(5, 5))
            plt.imshow(img)
            plt.axis('off')  # Hide axes for better visual presentation
            plt.title(f"Recognized Instrument: {predicted_class}", fontsize=16)
            plt.show()
        except SyntaxError:
            print(f"File at {image_path} is not a valid PNG file.")
    else:
        print(f"Image for instrument '{predicted_class}' not found.")

display_instrument_image(predicted_class)
